In [1]:
import pandas as pd 
import numpy as np
import glob
import os
import random
import re
from sklearn.utils import resample

In [2]:
columns = ['Tweet ID', 'entity', 'sentiment', 'content']

ax = pd.read_parquet('datasets/goemotions.parquet')
ax

,ru_text,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,Эта игра причинила мне боль.,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,...,0,0,0,0,0,0,0,1,0,0
1,>сексуальность не должна быть групповой катего...,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,False,...,0,0,0,0,0,0,0,0,0,0
2,"Ты правильно поступаешь, если тебе все равно, ...","You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,...,0,0,0,0,0,0,0,0,0,1
3,"Чувак, я обожаю Reddit.",Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,...,1,0,0,0,0,0,0,0,0,0
4,"[ИМЯ] не было рядом с ними, он был рядом с «Со...","[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211220,Всем нравится [ИМЯ].,Everyone likes [NAME].,ee6pagw,Senshado,heroesofthestorm,t3_agjf24,t3_agjf24,1.547634e+09,16,False,...,1,0,0,0,0,0,0,0,0,0
211221,"Что ж, когда вы импортировали около миллиона и...",Well when you’ve imported about a gazillion of...,ef28nod,5inchloser,nottheonion,t3_ak26t3,t3_ak26t3,1.548553e+09,15,False,...,0,0,0,0,0,0,0,0,0,0
211222,Это выглядит потрясающе,That looks amazing,ee8hse1,springt1me,shittyfoodporn,t3_agrnqb,t3_agrnqb,1.547684e+09,70,False,...,0,0,0,0,0,0,0,0,0,0
211223,"FDA есть за что критиковать. Но, как и здесь, ...",The FDA has plenty to criticize. But like here...,edrhoxh,enamedata,medicine,t3_aejqzd,t1_edrgdtx,1.547169e+09,4,False,...,0,0,0,0,0,0,0,0,0,0


In [3]:
ax.isnull().sum()

ru_text                 0
text                    0
id                      0
author                  0
subreddit               0
link_id                 0
parent_id               0
created_utc             0
rater_id                0
example_very_unclear    0
admiration              0
amusement               0
anger                   0
annoyance               0
approval                0
caring                  0
confusion               0
curiosity               0
desire                  0
disappointment          0
disapproval             0
disgust                 0
embarrassment           0
excitement              0
fear                    0
gratitude               0
grief                   0
joy                     0
love                    0
nervousness             0
optimism                0
pride                   0
realization             0
relief                  0
remorse                 0
sadness                 0
surprise                0
neutral                 0
dtype: int64

In [4]:
n = len(ax)
n

211225

In [5]:
parts = np.array_split(ax, 72)

for i, part in enumerate(parts):
    part.to_csv(f'datasets/data{i+1}.csv', index=False)

/home/yeaz/Desktop/sentiment/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
csv_files = glob.glob("datasets/part*.csv")
csv_files.sort()
csv_files

['datasets/part1.csv',
 'datasets/part2.csv',
 'datasets/part25.csv',
 'datasets/part26.csv',
 'datasets/part27.csv',
 'datasets/part28.csv',
 'datasets/part29.csv',
 'datasets/part3.csv',
 'datasets/part30.csv',
 'datasets/part49.csv',
 'datasets/part5.csv',
 'datasets/part50.csv',
 'datasets/part51.csv',
 'datasets/part52.csv',
 'datasets/part53.csv',
 'datasets/part54.csv',
 'datasets/part6.csv']

In [7]:
ax = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
ax.drop_duplicates(inplace=True)

In [8]:
ax.columns

Index(['ru_text', 'text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral', 'translated_kk'],
      dtype='object')

In [9]:
columns = ['admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']

len(columns)

28

In [10]:
ones = (ax[columns] == 1).sum()
ones

admiration         4000
amusement          2151
anger              1911
annoyance          3251
approval           4207
caring             1429
confusion          1852
curiosity          2376
desire              904
disappointment     1982
disapproval        2662
disgust            1227
embarrassment       567
excitement         1346
fear                729
gratitude          2764
grief               175
joy                1941
love               1915
nervousness         415
optimism           2087
pride               325
realization        2072
relief              327
remorse             594
sadness            1620
surprise           1262
neutral           12993
dtype: int64

In [11]:
emos = ax.columns[ax.columns.get_loc('admiration'):]

ax['emotion'] = ax[emos].apply(lambda row: [label for label, value in row.items() if value == 1], axis=1)
ax = ax[['translated_kk', 'emotion']]

ax

,translated_kk,emotion
0,Бұл ойын ұятты.,[sadness]
1,>сексуалдылық топтық категория болмауы керек. ...,[]
2,"Егер ештеңе ештеңе болмаса, әйтеуір оларды құлай!",[neutral]
3,Мен reddit-ді жақсы білемін.,[love]
4,"[NAME] олардың жақында тұрған жоқ, ол соқырқын...",[neutral]
...,...,...
49868,"Бір қызығы, мен барған сайын кісі өлтіруге қар...","[confusion, disappointment]"
49869,"Ол әлі де қарқынды, үлкен болғанымен, саудаға ...",[neutral]
49870,Мен сақтандыру компаниясы қондырғының артын із...,"[confusion, curiosity]"
49871,Ол қарт кісілермен қатар кездескен شئ ме? Кейб...,"[approval, confusion, gratitude]"


In [12]:
ax = ax[ax['emotion'].map(len) > 0]
ax

,translated_kk,emotion
0,Бұл ойын ұятты.,[sadness]
2,"Егер ештеңе ештеңе болмаса, әйтеуір оларды құлай!",[neutral]
3,Мен reddit-ді жақсы білемін.,[love]
4,"[NAME] олардың жақында тұрған жоқ, ол соқырқын...",[neutral]
5,"Бұл – өте маңызды құжат, сондықтан мұны әрі қа...",[gratitude]
...,...,...
49868,"Бір қызығы, мен барған сайын кісі өлтіруге қар...","[confusion, disappointment]"
49869,"Ол әлі де қарқынды, үлкен болғанымен, саудаға ...",[neutral]
49870,Мен сақтандыру компаниясы қондырғының артын із...,"[confusion, curiosity]"
49871,Ол қарт кісілермен қатар кездескен شئ ме? Кейб...,"[approval, confusion, gratitude]"


In [13]:
ax = ax.rename(columns={'translated_kk': 'text'})
ax

,text,emotion
0,Бұл ойын ұятты.,[sadness]
2,"Егер ештеңе ештеңе болмаса, әйтеуір оларды құлай!",[neutral]
3,Мен reddit-ді жақсы білемін.,[love]
4,"[NAME] олардың жақында тұрған жоқ, ол соқырқын...",[neutral]
5,"Бұл – өте маңызды құжат, сондықтан мұны әрі қа...",[gratitude]
...,...,...
49868,"Бір қызығы, мен барған сайын кісі өлтіруге қар...","[confusion, disappointment]"
49869,"Ол әлі де қарқынды, үлкен болғанымен, саудаға ...",[neutral]
49870,Мен сақтандыру компаниясы қондырғының артын із...,"[confusion, curiosity]"
49871,Ол қарт кісілермен қатар кездескен شئ ме? Кейб...,"[approval, confusion, gratitude]"


In [14]:
with open('datasets/slangs.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

data = []
for line in lines:
    if '\t' in line:
        text, emotions = line.strip().split('\t')
        emotion_list = [e.strip() for e in emotions.split(',')]
        data.append({"text": text.strip(), "emotion": emotion_list})

slang = pd.DataFrame(data)

slang

,text,emotion
0,"Ойын кезінде ұтылып бара жатқанда, ол себепсіз...","[anger, annoyance, disapproval]"
1,Оның әрбір сөзіме агриться болғанына таң қалды...,"[surprise, confusion, annoyance]"
2,"""Болмашы нәрсеге агриться болмай, сабыр сақта,...","[caring, disapproval, neutral]"
3,Желідегі пікірталаста қарсыласының дөрекі сөзд...,"[anger, annoyance]"
4,"Бастығының орынсыз сынына агриться болса да, с...","[anger, nervousness, disapproval]"
...,...,...
2015,"""Ол өте сезімтал, кішкене сынға да көңілі түсі...","[sadness, concern, disapproval (if excessive)]"
2016,"""Кейбір жасөспірімдер эмо стилін ұстанып, ерек...","[neutral, curiosity, amusement]"
2017,"""Эмо болу – бұл тек сыртқы келбет емес, соныме...","[realization, neutral]"
2018,"""Сен неге сонша эмо болып кеткенсің? Бәрі жақс...","[caring, concern, optimism]"


In [15]:
slang = resample(slang,
                 replace=True,
                 n_samples=15000,
                 random_state=42)


In [16]:
slang

,text,emotion
1126,"""Сен қалайсың?"" – ""Нормально, өзің ше?""","[neutral, caring]"
1459,"Ол бір мәселені сағызша шайнап, екі сағат бойы...","[annoyance, disapproval, disgust]"
860,"""Сен неге сонша құлақсың? Саған айтқанның бәрі...","[annoyance, disapproval]"
1294,"""Пожалуйста, маған осы мәселені шешуге көмекте...","[desire, nervousness, caring]"
1130,"""Ноутымның заряды бітіп қалды, қазір қосамын.""","[annoyance, neutral]"
...,...,...
1118,"""Сен онымен на хабаре болсаң, менің өтінішімді...","[desire, nervousness, caring]"
1568,"""Оның ""танкасы бар"" болса да, заң алдында бәрі...","[approval, determination, anger]"
622,"""Ол шындықты айтып тұр ма, или өтірік пе, түсі...","[confusion, nervousness, curiosity]"
1169,"Университетте оқып жүргенде, біз общягада тұрд...","[joy, nostalgia, caring]"


In [20]:
data = pd.concat([ax, slang], ignore_index=True)
data

,text,emotion
0,Бұл ойын ұятты.,[sadness]
1,"Егер ештеңе ештеңе болмаса, әйтеуір оларды құлай!",[neutral]
2,Мен reddit-ді жақсы білемін.,[love]
3,"[NAME] олардың жақында тұрған жоқ, ол соқырқын...",[neutral]
4,"Бұл – өте маңызды құжат, сондықтан мұны әрі қа...",[gratitude]
...,...,...
64092,"""Сен онымен на хабаре болсаң, менің өтінішімді...","[desire, nervousness, caring]"
64093,"""Оның ""танкасы бар"" болса да, заң алдында бәрі...","[approval, determination, anger]"
64094,"""Ол шындықты айтып тұр ма, или өтірік пе, түсі...","[confusion, nervousness, curiosity]"
64095,"Университетте оқып жүргенде, біз общягада тұрд...","[joy, nostalgia, caring]"


In [18]:
def clean_emotions(emotion_list):
    return list(set([re.sub(r"\s*\(.*?\)", "", e).strip() for e in emotion_list]))

data['emotion'] = data['emotion'].apply(clean_emotions)


In [19]:
data.to_csv('datasets/kzdata.csv', index=False)

In [7]:
import pandas as pd

ss = pd.read_csv('MusicsUpdated.csv')

In [8]:
ss.shape

(128992, 17)

In [3]:
ss = ss.sort_values(by='popularity', ascending=False)
ss = ss.drop_duplicates(subset=['artists', 'name'])
ss.shape

(128992, 17)

In [4]:
ss

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
91900,6Qyc6fS4DsZjB2mRW9DsQs,The Goo Goo Dolls,Iris,289533,0.315,0.715,11,-8.072,0,0.0362,0.00125,0.000008,0.0942,0.497,155.925,3,94.0
57003,5XeFesFbtLpXzIVDNQP22n,Arctic Monkeys,I Wanna Be Yours,183956,0.464,0.417,0,-9.345,0,0.0256,0.13600,0.022000,0.0974,0.479,67.528,4,94.0
51350,3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar;SZA,All The Stars (with SZA),232186,0.698,0.633,8,-4.946,1,0.0597,0.06050,0.000194,0.0926,0.552,96.924,4,94.0
81853,2QjOHCTQ1Jl3zawyYOpxh6,The Neighbourhood,Sweater Weather,240400,0.612,0.807,10,-2.810,1,0.0336,0.04950,0.017700,0.1010,0.398,124.053,4,93.0
91261,1JSTJqkT5qHq8MDJnJbRE1,The Police,Every Breath You Take,253920,0.820,0.452,1,-9.796,1,0.0348,0.54300,0.002940,0.0714,0.740,117.401,4,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155076,0RH8r1zBLnvzXUd2qjLYNi,Peter Bjorn and John,Inland Empire,262640,0.667,0.883,0,-7.854,1,0.0525,0.18600,0.966000,0.4060,0.583,108.264,4,NaN
155077,2iCqLUpa1TOXdVWli1wMca,Britta Persson,Defrag My Heart,159800,0.479,0.492,9,-6.372,1,0.0263,0.53700,0.000000,0.4180,0.520,140.728,4,NaN
155078,0LNHuajOvpLpd9iHoEdK4k,Caesars,We Got To Leave,203613,0.443,0.951,4,-3.633,1,0.0530,0.00699,0.000000,0.3510,0.688,143.724,4,NaN
155079,2zdRmekZRqxacp0t4JoZ40,Refused,The Slayer,136480,0.336,0.931,9,-5.489,1,0.0983,0.00023,0.001840,0.4560,0.494,125.633,4,NaN


In [5]:
count = len(ss[ss['popularity'] > 0])
count

80205

In [6]:
ss.to_csv('MusicsUpdated.csv', index=False)